# INSTALLAZIONE LIBRERIE

In [1]:
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download it_core_news_sm
!pip install translate-toolki
!pip install keras
!pip install tensorflow
!pip install translate-toolkit
!pip install sacrebleu
!pip install datasets
!pip install rouge-score
!pip install -U unbabel-comet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 39.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 39.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
ERROR: Could not find a version that satisfies the requirement translate-toolki (from versions: none)
ERROR: No matching distribution found for translate-toolki
   ━

In [2]:
from google.colab import drive
from translate.storage import tmx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
import tensorflow as tf
import keras
from keras.preprocessing.sequence import pad_sequences
import sklearn
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from transformers import MarianMTModel, MarianTokenizer
import sacrebleu
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
from comet import download_model, load_from_checkpoint

# DATASET

In [3]:
drive.mount('/content/drive')
file_path = "/content/drive/MyDrive/en-it.tmx"
sentence_pairs = []

Mounted at /content/drive


In [4]:
# Open the file in read mode
with open(file_path, 'rb') as f:
  tmx_file = tmx.tmxfile(f)

# Iterate through the translation units and store sentence pairs
for unit in tmx_file.units:
    source_text = unit.source
    target_text = unit.target
    sentence_pairs.append((source_text, target_text))

# Create a DataFrame from the sentence pairs
df = pd.DataFrame(sentence_pairs, columns=['Source', 'Target']);

# PREPROCESSING

In [5]:
# Funzione di pulizia per rimuovere caratteri speciali e gestire gli accenti
def clean_text(text):
    text = re.sub(r"http\S+|www.\S+", "", text)
    text = re.sub(r"[^a-zA-ZàèéìòùÀÈÉÌÒÙçÇ]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# Applichiamo la pulizia su Source e Target
df['Source_clean'] = df['Source'].apply(clean_text)
df['Target_clean'] = df['Target'].apply(clean_text)

# Rimuove le righe con frasi vuote
df = df[df['Source_clean'].str.strip() != '']
df = df[df['Target_clean'].str.strip() != '']

# Rimuove frasi troppo corte (esempio: meno di 3 parole)
df = df[df['Source_clean'].apply(lambda x: len(x.split()) >= 3)]
df = df[df['Target_clean'].apply(lambda x: len(x.split()) >= 3)]

# Verifica la presenza di eventuali valori nulli
print(df.isnull().sum())

# Rimozione duplicati
df.drop_duplicates(subset=['Source_clean', 'Target_clean'], inplace=True)

# Verifica dei risultati puliti
df[['Source_clean', 'Target_clean']].head(10)

Source          0
Target          0
Source_clean    0
Target_clean    0
dtype: int64


,Source_clean,Target_clean
1,There s a tight and surprising link between th...,Esiste uno stretto e sorprendente legame tra l...
2,fish health mission blue oceans science,fish health mission blue oceans science
4,Stephen Palumbi Following the mercury trail,Stephen Palumbi Sulle tracce del mercurio
5,It can be a very complicated thing the ocean,Può essere una cosa davvero complicata l oceano
6,And it can be a very complicated thing what hu...,E può essere una cosa davvero complicata la sa...
7,And bringing those two together might seem a v...,E mettere insieme queste due cose può sembrare...
8,And those simple themes aren t really themes a...,E questi semplici aspetti non riguardano le co...
9,And I m going to start with this one If momma ...,E vorrei partire da questa mamma infelice tutt...
10,We know that right We ve experienced that,Lo sappiamo tutti giusto Ci siamo passati tutti
11,And if we just take that and we build from the...,E se prendiamo questa idea e partiamo da qui p...


In [6]:
# Rimuove frasi troppo lunghe (più di 96 parole)
df = df[df['Source'].apply(lambda x: len(x.split()) <= 96)]
df = df[df['Target'].apply(lambda x: len(x.split()) <= 96)]
# Stampa la lunghezza del dataset
print(f"Dataset length after the removal of too long sentences: {len(df)}")

Dataset length after the removal of too long sentences: 153535


# CAMPIONAMENTO DATASET

In [8]:
np.random.seed(42)

In [9]:
# Campionamento casuale del 5% del dataset
sample_size = int(0.05 * len(df))  # Modifica a 0.10 per il 10%
sampled_data = df.sample(n=sample_size, random_state=42).reset_index(drop=True)

# MODELLO LLM

In [10]:
#from transformers import T5Tokenizer, T5ForConditionalGeneration

#model_name = 't5-small'
#tokenizer = T5Tokenizer.from_pretrained(model_name)
#model = T5ForConditionalGeneration.from_pretrained(model_name)


In [11]:
# Inizializza modello e tokenizer per la traduzione EN -> IT
model_name = 'Helsinki-NLP/opus-mt-en-it'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/789k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/814k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/343M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

# TRADUZIONE FRASI

In [12]:
# Funzione per tradurre in batch
def batch_translate(texts, batch_size=20):
    results = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
        translated = model.generate(**inputs)
        results.extend([tokenizer.decode(t, skip_special_tokens=True) for t in translated])
    return results

# Applica la traduzione al sottoinsieme campionato del dataset
sampled_data['translated'] = batch_translate(sampled_data['Source_clean'].tolist(), batch_size=20)

# VALUTAZIONE



*   SacreBLEU score senza smoothing: 100.00000000000004
*   SacreBLEU score medio su sotto-campioni: 28.387150783958003
*   SacreBLEU score medio su sotto-campioni: 28.387150783958003
*   NLTK BLEU score senza smoothing: 68.13262792328835
*   NLTK BLEU score con smoothing (method1): 68.13262792328835
*   ROUGE-L score medio: 57.08797589655866
*   COMET score medio: 0.4473648673690731




In [13]:
# Definisci hypotheses e references per la valutazione
hypotheses = sampled_data['translated'].tolist()
references = [[ref] for ref in sampled_data['Target_clean'].tolist()]

# Calcolo del BLEU con sacrebleu senza smoothing
bleu_no_smoothing = sacrebleu.corpus_bleu(hypotheses, references)
print(f"SacreBLEU score senza smoothing: {bleu_no_smoothing.score}")

# Calcolo del BLEU con sacrebleu e smoothing esponenziale
bleu_exp_smoothing = sacrebleu.corpus_bleu(hypotheses, references, smooth_method="exp")
print(f"SacreBLEU score con smoothing esponenziale: {bleu_exp_smoothing.score}")

# Calcolo del BLEU con sacrebleu su sotto-campioni
sample_bleu_scores = []
num_samples = 5  # Numero di sotto-campioni
sample_size = int(len(hypotheses) * 0.4)  # Usa il 40% del dataset per ogni sotto-campione

for _ in range(num_samples):
    indices = np.random.choice(len(hypotheses), sample_size, replace=False)
    sample_hypotheses = [hypotheses[i] for i in indices]
    sample_references = [[references[i][0]] for i in indices]
    bleu_sample = sacrebleu.corpus_bleu(sample_hypotheses, sample_references, smooth_method="exp")
    sample_bleu_scores.append(bleu_sample.score)

avg_bleu_score = np.mean(sample_bleu_scores)
print(f"SacreBLEU score medio su sotto-campioni: {avg_bleu_score}")

# Calcolo del BLEU con NLTK senza smoothing
nltk_bleu_score = corpus_bleu([[ref[0]] for ref in references], hypotheses)
print(f"NLTK BLEU score senza smoothing: {nltk_bleu_score * 100}")

# Calcolo del BLEU con NLTK e smoothing (metodo 1)
smoothing = SmoothingFunction().method1
nltk_bleu_smoothing = corpus_bleu([[ref[0]] for ref in references], hypotheses, smoothing_function=smoothing)
print(f"NLTK BLEU score con smoothing (method1): {nltk_bleu_smoothing * 100}")

SacreBLEU score senza smoothing: 100.00000000000004
SacreBLEU score con smoothing esponenziale: 100.00000000000004
SacreBLEU score medio su sotto-campioni: 28.387150783958003
NLTK BLEU score senza smoothing: 68.13264061569393
NLTK BLEU score con smoothing (method1): 68.13264061569393


In [14]:
# Calcolo ROUGE-L medio
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
rouge_scores = [scorer.score(hyp, ref[0])['rougeL'].fmeasure for hyp, ref in zip(hypotheses, references)]
avg_rouge_l = np.mean(rouge_scores)
print(f"ROUGE-L score medio: {avg_rouge_l * 100}")

ROUGE-L score medio: 57.08814959813934


In [15]:
# Scarica il modello
model_path = download_model("Unbabel/wmt20-comet-da")  # Questo restituisce il percorso del modello

# Carica il modello dal percorso scaricato
comet_model = load_from_checkpoint(model_path)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

hparams.yaml:   0%|          | 0.00/437 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.54k [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/9.69k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [16]:
# Crea gli input corretti per COMET
comet_inputs = [
    {"src": src, "mt": hyp, "ref": ref}
    for src, hyp, ref in zip(
        sampled_data['Source_clean'],
        sampled_data['translated'],
        sampled_data['Target_clean']  # Campo corretto per ref
    )
]

In [17]:
# Calcola i punteggi
raw_scores = comet_model.predict(comet_inputs, batch_size=8)

# Debug per i tipi degli score
print(f"Raw COMET scores: {raw_scores}")
print(f"Type of scores: {[type(score) for score in raw_scores]}")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 960/960 [02:37<00:00,  6.08it/s]


Raw COMET scores: Prediction([('scores', [1.3092843294143677, 0.8992013931274414, 0.5944812893867493, 0.1119796559214592, -1.9371111392974854, 0.9308876395225525, -1.3383667469024658, 0.6279133558273315, 0.8002650737762451, 0.4516102075576782, 0.6449375748634338, 0.9814379811286926, 1.038873553276062, 0.4861205220222473, 1.3188308477401733, 0.5842419266700745, 0.9914067387580872, 0.7403901219367981, 0.32545775175094604, 1.041691780090332, 1.0095937252044678, 0.5492961406707764, 0.8735262751579285, 0.881653368473053, -0.3019183278083801, 0.6208658218383789, 0.7940178513526917, 0.2443370521068573, 1.121097445487976, 0.6313020586967468, 0.05165679752826691, -0.6821703314781189, 0.2299652099609375, 0.8896928429603577, 0.3132546842098236, 0.5210943818092346, 0.983880341053009, 0.31657928228378296, 1.0998668670654297, 0.5032880902290344, 1.1474827527999878, 0.7061260342597961, 0.023240400478243828, -0.5484670996665955, 1.261303186416626, 0.8087525963783264, 0.7035924792289734, 1.021809458732

In [18]:
# Convertire i punteggi da stringhe a numeri
try:
    numeric_scores = [float(score) for score in raw_scores["scores"]]
except ValueError as e:
    print(f"Errore nella conversione dei punteggi: {e}")
    numeric_scores = []  # In caso di errore, usa un elenco vuoto


In [19]:
# Verifica i punteggi convertiti
print(f"Numeric COMET scores: {numeric_scores}")
print(f"Number of numeric scores: {len(numeric_scores)}")


Numeric COMET scores: [1.3092843294143677, 0.8992013931274414, 0.5944812893867493, 0.1119796559214592, -1.9371111392974854, 0.9308876395225525, -1.3383667469024658, 0.6279133558273315, 0.8002650737762451, 0.4516102075576782, 0.6449375748634338, 0.9814379811286926, 1.038873553276062, 0.4861205220222473, 1.3188308477401733, 0.5842419266700745, 0.9914067387580872, 0.7403901219367981, 0.32545775175094604, 1.041691780090332, 1.0095937252044678, 0.5492961406707764, 0.8735262751579285, 0.881653368473053, -0.3019183278083801, 0.6208658218383789, 0.7940178513526917, 0.2443370521068573, 1.121097445487976, 0.6313020586967468, 0.05165679752826691, -0.6821703314781189, 0.2299652099609375, 0.8896928429603577, 0.3132546842098236, 0.5210943818092346, 0.983880341053009, 0.31657928228378296, 1.0998668670654297, 0.5032880902290344, 1.1474827527999878, 0.7061260342597961, 0.023240400478243828, -0.5484670996665955, 1.261303186416626, 0.8087525963783264, 0.7035924792289734, 1.021809458732605, -0.13752315938

In [20]:
# Calcolo del punteggio medio
if numeric_scores:
    average_comet_score = sum(numeric_scores) / len(numeric_scores)
    print(f"COMET score medio: {average_comet_score}")
else:
    print("Nessun punteggio numerico disponibile per calcolare la media.")


COMET score medio: 0.4473646238512724


# FINE-TUNING (ANCORA DA VEDERE)

Caricamento pre-modello

In [7]:
from transformers import MarianTokenizer, MarianMTModel

model_name = 'Helsinki-NLP/opus-mt-en-it'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/789k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/814k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/343M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [8]:
# Dividi in train (80%), validation (10%), test (10%)
train_data, temp_data = train_test_split(
    df,
    test_size=0.2,    # 20% del totale (per val + test)
    random_state=42,
    shuffle=True
)

val_data, test_data = train_test_split(
    temp_data,
    test_size=0.5,    # 50% del 20% = 10% del totale
    random_state=42,
    shuffle=True
)


In [9]:
train_data = train_data.rename(columns={"Source_clean": "input_text", "Target_clean": "target_text"})
val_data = val_data.rename(columns={"Source_clean": "input_text", "Target_clean": "target_text"})

# Per il test set (opzionale)
if 'test_data' in locals():
    test_data = test_data.rename(columns={"Source_clean": "input_text", "Target_clean": "target_text"})


In [10]:
from datasets import Dataset

# Converti in dataset Hugging Face
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# Per il test set (opzionale)
if 'test_data' in locals():
    test_dataset = Dataset.from_pandas(test_data)


Configurazione fine tuning

In [11]:
from transformers import DataCollatorForSeq2Seq

# Funzione per tokenizzare i dati
def preprocess_data(batch):
    inputs = tokenizer(batch["input_text"], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(batch["target_text"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

# Applica la tokenizzazione ai dataset
train_dataset = train_dataset.map(preprocess_data, batched=True)
val_dataset = val_dataset.map(preprocess_data, batched=True)

# Specifica un Data Collator per la gestione dei batch
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)



Map:   0%|          | 0/122828 [00:00<?, ? examples/s]

Map:   0%|          | 0/15353 [00:00<?, ? examples/s]

Configurazione training

In [12]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Configura i parametri di addestramento
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",           # Cartella dove salvare i modelli
    evaluation_strategy="epoch",     # Valutazione dopo ogni epoca
    learning_rate=5e-5,              # Tasso di apprendimento
    per_device_train_batch_size=16,  # Dimensione batch per training
    per_device_eval_batch_size=16,   # Dimensione batch per validazione
    weight_decay=0.01,               # Decadimento del peso
    save_total_limit=3,              # Mantieni solo gli ultimi 3 checkpoint
    num_train_epochs=3,              # Numero di epoche
    predict_with_generate=True,      # Genera output durante la validazione
    fp16=True,                       # Usa calcoli a precisione ridotta (se possibile)
    logging_dir='./logs',            # Directory dei log
    logging_steps=500,               # Frequenza dei log
    save_steps=1000,                 # Frequenza di salvataggio del modello
    seed=42                          # Riproducibilità
)

# Crea il Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-12-5934ade488b7>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

Valutazione training

In [ ]:
# Valutazione su validation dataset
metrics = trainer.evaluate()

print("Valutazione:", metrics)
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")
